In [106]:
from utils import spark
from pyspark.sql import functions as F
from pyspark.sql.types import *
import seaborn as sns
import matplotlib.pyplot as plt

# Load data

In [107]:
%%time
rating_df=spark.read.csv('data/ratings.csv',header=True)
tags_df=spark.read.csv("data/tags.csv",header=True)
title_df=spark.read.csv("data/movies.csv",header=True)

CPU times: user 2.31 ms, sys: 4.12 ms, total: 6.44 ms
Wall time: 633 ms


In [108]:
tags_df.columns

['userId', 'movieId', 'tag', 'timestamp']

In [109]:
%%time
print("jumlah User yang memberikan rating:",rating_df.select("userId").distinct().count())
print("Jumlah tidak ada Movies:",rating_df.select("movieId").distinct().count())
print("jumlah yang tidak ada tag ratings:",rating_df.select("rating").count())
print("Jumlah user yang tidak komentar:",tags_df.select("tag").distinct().count())
print("Jumlah film yang tidak komentar:",tags_df.select("movieId").distinct().count())
print("pesentasi pengguna yang berkomentar:",
      tags_df.select("tag").distinct().count()
      / rating_df.select("userId").distinct().count()*100)
print("persentasi film yang tidal berkomentar oleh user:",tags_df.select('movieId').distinct().count()
      / rating_df.select("movieId").distinct().count())

jumlah User yang memberikan rating: 162541


Jumlah tidak ada Movies: 59047


jumlah yang tidak ada tag ratings: 25000095


Jumlah user yang tidak komentar: 73051


Jumlah film yang tidak komentar: 45251


pesentasi pengguna yang berkomentar: 44.94312204305375


persentasi film yang tidal berkomentar oleh user: 0.7663556150185445
CPU times: user 74.3 ms, sys: 4.31 ms, total: 78.6 ms
Wall time: 1min 37s


In [110]:
rating_df=rating_df.drop('timestamp')
tags_df=tags_df.drop('timestamp')

In [111]:
title_df=title_df.withColumn("Year",F.regexp_extract("title",r'\((\d{4})\)',1))
title_df.show(n=3,truncate=False)

+-------+-----------------------+-------------------------------------------+----+
|movieId|title                  |genres                                     |Year|
+-------+-----------------------+-------------------------------------------+----+
|1      |Toy Story (1995)       |Adventure|Animation|Children|Comedy|Fantasy|1995|
|2      |Jumanji (1995)         |Adventure|Children|Fantasy                 |1995|
|3      |Grumpier Old Men (1995)|Comedy|Romance                             |1995|
+-------+-----------------------+-------------------------------------------+----+
only showing top 3 rows



In [112]:
sss.show()

+--------------------+----+---------+---------+--------+------+--------+------+------+------+------+-------+
|               title|year|   genre1|   genre2|  genre3|genre4|  genre5|genre6|genre7|genre8|genre9|genre10|
+--------------------+----+---------+---------+--------+------+--------+------+------+------+------+-------+
|    Toy Story (1995)|1995|Adventure|Animation|Children|Comedy| Fantasy|  null|  null|  null|  null|   null|
|      Jumanji (1995)|1995|Adventure| Children| Fantasy|  null|    null|  null|  null|  null|  null|   null|
|Grumpier Old Men ...|1995|   Comedy|  Romance|    null|  null|    null|  null|  null|  null|  null|   null|
|Waiting to Exhale...|1995|   Comedy|    Drama| Romance|  null|    null|  null|  null|  null|  null|   null|
|Father of the Bri...|1995|   Comedy|     null|    null|  null|    null|  null|  null|  null|  null|   null|
|         Heat (1995)|1995|   Action|    Crime|Thriller|  null|    null|  null|  null|  null|  null|   null|
|      Sabrina (199

In [113]:
from pyspark.sql.functions import col, split, expr
from pyspark.sql.types import IntegerType

# Separate genre into each column
split_cols = [split(col('genres'), '\|').getItem(i).alias(f'genre{i+1}') for i in range(10)]
split_cols.insert(0, col('movieId'))  # Add 'movieId' as the first column
sss = title_df.select(*split_cols, 'title', 'year')

# Count genres (non-zero values)
genre_columns = [col(f'genre{i+1}') for i in range(10)]
title_df = sss.withColumn('genre_count', sum((col_name != 'null').cast(IntegerType()) for col_name in genre_columns))

title_df.show(3)


+-------+---------+---------+--------+------+-------+------+------+------+------+-------+--------------------+----+-----------+
|movieId|   genre1|   genre2|  genre3|genre4| genre5|genre6|genre7|genre8|genre9|genre10|               title|year|genre_count|
+-------+---------+---------+--------+------+-------+------+------+------+------+-------+--------------------+----+-----------+
|      1|Adventure|Animation|Children|Comedy|Fantasy|  null|  null|  null|  null|   null|    Toy Story (1995)|1995|       null|
|      2|Adventure| Children| Fantasy|  null|   null|  null|  null|  null|  null|   null|      Jumanji (1995)|1995|       null|
|      3|   Comedy|  Romance|    null|  null|   null|  null|  null|  null|  null|   null|Grumpier Old Men ...|1995|       null|
+-------+---------+---------+--------+------+-------+------+------+------+------+-------+--------------------+----+-----------+
only showing top 3 rows



In [114]:
rating_df.show(3)

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|    296|   5.0|
|     1|    306|   3.5|
|     1|    307|   5.0|
+------+-------+------+
only showing top 3 rows



In [115]:
from pyspark.sql.functions import avg, count

# Avg movie ratings by movieId and count
rating_avg = rating_df.groupBy('movieId').agg(avg('rating').alias('avg_rating'))
rating_count = rating_df.groupBy('movieId').agg(count('rating').alias('rating_count'))

# Avg user ratings by userId and count
user_rating = rating_df.groupBy('userId').agg(avg('rating').alias('avg_rating'))
user_count = rating_df.groupBy('userId').agg(count('rating').alias('rating_count'))

# Convert to PySpark DataFrames
rating_avg = rating_avg.withColumnRenamed('movieId', 'movieId')
rating_count = rating_count.withColumnRenamed('movieId', 'movieId').withColumnRenamed('rating_count', 'rating_count')
user_rating = user_rating.withColumnRenamed('userId', 'userId')
user_count = user_count.withColumnRenamed('userId', 'userId').withColumnRenamed('rating_count', 'rating_count')

# Merge DataFrames
df_movie = rating_avg.join(rating_count, 'movieId', 'inner')
df_movie = df_movie.orderBy('avg_rating', ascending=True)
df_user = user_rating.join(user_count, 'userId', 'inner')
df_user = df_user.orderBy('avg_rating', ascending=True)

In [116]:
df_user.show(4)

+------+----------+------------+
|userId|avg_rating|rating_count|
+------+----------+------------+
| 10364|       0.5|          20|
| 61102|       0.5|          20|
| 38998|       0.5|          20|
| 63044|       0.5|          20|
+------+----------+------------+
only showing top 4 rows



In [117]:
df_movie.show(4)

+-------+----------+------------+
|movieId|avg_rating|rating_count|
+-------+----------+------------+
| 203264|       0.5|           1|
| 197471|       0.5|           1|
| 207475|       0.5|           1|
| 179185|       0.5|           1|
+-------+----------+------------+
only showing top 4 rows



In [118]:
title_df.columns

['movieId',
 'genre1',
 'genre2',
 'genre3',
 'genre4',
 'genre5',
 'genre6',
 'genre7',
 'genre8',
 'genre9',
 'genre10',
 'title',
 'year',
 'genre_count']

In [121]:
cols = ['movieId','genre1']
dfk = title_df.select(*[col(c) for c in cols])
gabung_tag=tags_df.join(dfk,on="movieId",how='inner')
gabung_tag.show()

+-------+------+--------------------+---------+
|movieId|userId|                 tag|   genre1|
+-------+------+--------------------+---------+
|    260|     3|             classic|   Action|
|    260|     3|              sci-fi|   Action|
|   1732|     4|         dark comedy|   Comedy|
|   1732|     4|      great dialogue|   Comedy|
|   7569|     4|    so bad it's good|   Action|
|  44665|     4|unreliable narrators|    Crime|
| 115569|     4|               tense|    Crime|
| 115713|     4|artificial intell...|    Drama|
| 115713|     4|       philosophical|    Drama|
| 115713|     4|               tense|    Drama|
| 148426|     4|    so bad it's good|    Drama|
| 164909|     4|              cliche|   Comedy|
| 164909|     4|             musical|   Comedy|
| 168250|     4|              horror|   Horror|
| 168250|     4|       unpredictable|   Horror|
|   2160|    19|Oscar (Best Suppo...|    Drama|
|   7099|    19|           adventure|Adventure|
|   7099|    19|               anime|Adv